# Model Zoo
> Base TTS Model test code for LatentForce VoiceStudio

## 0. Common Code

In [ ]:
import torch
from transformers import AutoTokenizer, AutoProcessor

from voicestudio.utils.audio_utils import show_waveform

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("INFO: Using device -", device)

## 1. Parler TTS

In [ ]:
from voicestudio.models.parler_tts import ParlerTTSForConditionalGeneration

In [ ]:
model_id = "parler-tts/parler-tts-mini-v1"

model = ParlerTTSForConditionalGeneration.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
prompt = "Hey, how are you doing today?"
#description = "Jon's voice is monotone yet slightly fast in delivery, with a very close recording that almost has no background noise."
description = "A female speaker delivers a slightly expressive and animated speech with a moderate speed and pitch. The recording is of very high quality, with the speaker's voice sounding clear and very close up."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().squeeze()

show_waveform(None, waveform=audio_arr, sr=model.config.sampling_rate)

In [ ]:
"""
아주 간단한 테스트: kwargs로 max_length 전달
"""

import torch
from transformers import AutoTokenizer
from voicestudio.models.parler_tts import ParlerTTSForConditionalGeneration

model_name = "parler-tts/parler-tts-mini-v1"
model = ParlerTTSForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(f"Default max_length: {model.generation_config.max_length}")
print(f"Default decoder_start_token_id: {model.generation_config.decoder_start_token_id}")
print(f"Default bos_token_id: {model.generation_config.bos_token_id}")
print(f"Num codebooks: {model.decoder.num_codebooks}")

description = "Jon's voice is monotone."
prompt = "Hey."

inputs = tokenizer(description, return_tensors="pt")
prompt_tokens = tokenizer(prompt, return_tensors="pt")

print(f"\nGenerating with max_length=100 as kwarg...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        prompt_input_ids=prompt_tokens.input_ids,
        max_length=100,  # kwargs로 전달
        do_sample=False
    )

actual_seq_len = outputs.shape[-1] // model.decoder.num_codebooks
print(f"\nOutput shape: {outputs.shape}")
print(f"Output total tokens: {outputs.shape[-1]}")
print(f"Actual sequence length (/ {model.decoder.num_codebooks} codebooks): {actual_seq_len}")
print(f"Expected: ~100")

if actual_seq_len > 500:
    print(f"❌ FAILED: Using default {actual_seq_len} instead of 100!")
else:
    print(f"✅ SUCCESS!")


## 2. Higgs Audio (v1/v2)

In [ ]:
from boson_multimodal.serve.serve_engine import HiggsAudioServeEngine, HiggsAudioResponse
from boson_multimodal.data_types import ChatMLSample, Message, AudioContent

import torch
import torchaudio
import time
import click

MODEL_PATH = "bosonai/higgs-audio-v2-generation-3B-base"
AUDIO_TOKENIZER_PATH = "bosonai/higgs-audio-v2-tokenizer"

system_prompt = (
    "Generate audio following instruction.\n\n<|scene_desc_start|>\nAudio is recorded from a quiet room.\n<|scene_desc_end|>"
)

messages = [
    Message(
        role="system",
        content=system_prompt,
    ),
    Message(
        role="user",
        content="The sun rises in the east and sets in the west. This simple fact has been observed by humans for thousands of years.",
    ),
]
device = "cuda" if torch.cuda.is_available() else "cpu"

serve_engine = HiggsAudioServeEngine(MODEL_PATH, AUDIO_TOKENIZER_PATH, device=device)

output: HiggsAudioResponse = serve_engine.generate(
    chat_ml_sample=ChatMLSample(messages=messages),
    max_new_tokens=1024,
    temperature=0.3,
    top_p=0.95,
    top_k=50,
    stop_strings=["<|end_of_text|>", "<|eot_id|>"],
)
torchaudio.save(f"output.wav", torch.from_numpy(output.audio)[None, :], output.sampling_rate)

In [ ]:
from voicestudio.boson_multimodal.serve.serve_engine import HiggsAudioServeEngine, HiggsAudioResponse
from voicestudio.boson_multimodal.data_types import ChatMLSample, Message
import torch
import torchaudio

MODEL_PATH = "bosonai/higgs-audio-v2-generation-3B-base"
AUDIO_TOKENIZER_PATH = "bosonai/higgs-audio-v2-tokenizer"

system_prompt = (
    "Generate audio following instruction.\n\n<|scene_desc_start|>\nAudio is recorded from a quiet room.\n<|scene_desc_end|>"
)

messages = [
    Message(role="system", content=system_prompt),
    Message(role="user", content="The sun rises in the east and sets in the west."),
]

device = "cuda" if torch.cuda.is_available() else "cpu"

# OLD: Use HiggsAudioServeEngine
serve_engine = HiggsAudioServeEngine(MODEL_PATH, AUDIO_TOKENIZER_PATH, device=device)

output: HiggsAudioResponse = serve_engine.generate(
    chat_ml_sample=ChatMLSample(messages=messages),
    max_new_tokens=1024,
    temperature=0.3,
    top_p=0.95,
    top_k=50,
    stop_strings=["<|end_of_text|>", "<|eot_id|>"],
)

torchaudio.save(f"output.wav", torch.from_numpy(output.audio)[None, :], output.sampling_rate)

In [ ]:
from voicestudio.boson_multimodal.model.higgs_audio import (
    HiggsAudioForConditionalGeneration,
    HiggsAudioProcessor
)
from voicestudio.boson_multimodal.data_types import ChatMLSample, Message
import torch
import torchaudio

MODEL_PATH = "bosonai/higgs-audio-v2-generation-3B-base"
AUDIO_TOKENIZER_PATH = "bosonai/higgs-audio-v2-tokenizer"

system_prompt = (
    "Generate audio following instruction.\n\n<|scene_desc_start|>\nAudio is recorded from a quiet room.\n<|scene_desc_end|>"
)

messages = [
    Message(role="system", content=system_prompt),
    Message(role="user", content="The sun rises in the east and sets in the west."),
]

device = "cuda" if torch.cuda.is_available() else "cpu"

# NEW: Load model and processor separately
model = HiggsAudioForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto"
).to(device)

# Set audio tokenizer
model.set_audio_tokenizer(AUDIO_TOKENIZER_PATH, device=device)

# Optional: Prepare KV caches for optimization
model.prepare_kv_caches()
if device == "cuda":
    model.enable_cuda_graphs()

# Load processor
processor = HiggsAudioProcessor.from_pretrained(
    MODEL_PATH,
    audio_tokenizer_path=AUDIO_TOKENIZER_PATH,
    device=device
)

# Process inputs
inputs = processor(ChatMLSample(messages=messages))

# Move to device
for k, v in inputs.items():
    if isinstance(v, torch.Tensor):
        inputs[k] = v.to(device)

# Generate
output = model.generate(
    **inputs,
    max_new_tokens=1024,
    temperature=0.3,
    top_p=0.95,
    top_k=50,
    return_audio_waveforms=True,  # Automatically decode audio
)

torchaudio.save("output.wav", torch.from_numpy(output.audio_waveforms)[None, :], output.sampling_rate)

In [ ]:
from voicestudio.boson_multimodal.model.higgs_audio import (
    HiggsAudioForConditionalGeneration,
    HiggsAudioProcessor
)
import torch
import torchaudio

MODEL_PATH = "bosonai/higgs-audio-v2-generation-3B-base"
AUDIO_TOKENIZER_PATH = "bosonai/higgs-audio-v2-tokenizer"

# System prompt
system_prompt = (
    "Generate audio following instruction.\n\n"
    "<|scene_desc_start|>\n"
    "Audio is recorded from a quiet room.\n"
    "<|scene_desc_end|>"
)

# Standard Transformers messages format (list of dicts)
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "The sun rises in the east and sets in the west. "
                                "This simple fact has been observed by humans for thousands of years."},
]

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model
model = HiggsAudioForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    dtype="auto"
).to(device)

# Set audio tokenizer
model.set_audio_tokenizer(AUDIO_TOKENIZER_PATH, device=device)

# Optionally prepare KV caches and enable CUDA graphs for optimization
model.prepare_kv_caches()
if device == "cuda":
    model.enable_cuda_graphs()

# Load processor
processor = HiggsAudioProcessor.from_pretrained(
    MODEL_PATH,
    audio_tokenizer_path=AUDIO_TOKENIZER_PATH,
    device=device
)

# Process inputs using standard messages format
batch_input = processor(messages)

# Move to device (HiggsAudioBatchInput has attributes, not dict keys)
batch_input.input_ids = batch_input.input_ids.to(device)
batch_input.attention_mask = batch_input.attention_mask.to(device)
if batch_input.audio_features is not None:
    batch_input.audio_features = batch_input.audio_features.to(device)
if batch_input.audio_feature_attention_mask is not None:
    batch_input.audio_feature_attention_mask = batch_input.audio_feature_attention_mask.to(device)
if batch_input.audio_in_ids is not None:
    batch_input.audio_in_ids = batch_input.audio_in_ids.to(device)
if batch_input.audio_in_ids_start is not None:
    batch_input.audio_in_ids_start = batch_input.audio_in_ids_start.to(device)
if batch_input.audio_out_ids is not None:
    batch_input.audio_out_ids = batch_input.audio_out_ids.to(device)
if batch_input.audio_out_ids_start is not None:
    batch_input.audio_out_ids_start = batch_input.audio_out_ids_start.to(device)

# Generate (convert dataclass to kwargs)
from dataclasses import asdict
output = model.generate(
    **asdict(batch_input),
    max_new_tokens=1024,
    temperature=0.3,
    top_p=0.95,
    top_k=50,
    return_audio_waveforms=True,
)

# Save audio
torchaudio.save(
    "output.wav",
    torch.from_numpy(output.audio_waveforms)[None, :],
    output.sampling_rate
)

print("Audio saved to output.wav")

## 3. Spark TTS

## 4. Dia 1

In [ ]:
from transformers import AutoProcessor, DiaForConditionalGeneration

torch_device = "cuda"
model_checkpoint = "nari-labs/Dia-1.6B-0626"

text = ["[S1] Dia is an open weights text to dialogue model."]
processor = AutoProcessor.from_pretrained(model_checkpoint)
inputs = processor(text=text, padding=True, return_tensors="pt").to(torch_device)

model = DiaForConditionalGeneration.from_pretrained(model_checkpoint).to(torch_device)
outputs = model.generate(**inputs, max_new_tokens=256)  # corresponds to around ~2s

# save audio to a file
outputs = processor.batch_decode(outputs)
processor.save_audio(outputs, "example.wav")

In [ ]:
from datasets import load_dataset, Audio
from transformers import AutoProcessor, DiaForConditionalGeneration

torch_device = "cuda"
model_checkpoint = "nari-labs/Dia-1.6B-0626"

ds = load_dataset("hf-internal-testing/dailytalk-dummy", split="train")
ds = ds.cast_column("audio", Audio(sampling_rate=44100))
audio = ds[-1]["audio"]["array"]
# text is a transcript of the audio + additional text you want as new audio
text = ["[S1] I know. It's going to save me a lot of money, I hope. [S2] I sure hope so for you."]

processor = AutoProcessor.from_pretrained(model_checkpoint)
inputs = processor(text=text, audio=audio, padding=True, return_tensors="pt").to(torch_device)
prompt_len = processor.get_audio_prompt_len(inputs["decoder_attention_mask"])

model = DiaForConditionalGeneration.from_pretrained(model_checkpoint).to(torch_device)
outputs = model.generate(**inputs, max_new_tokens=256)  # corresponds to around ~2s

# retrieve actually generated audio and save to a file
outputs = processor.batch_decode(outputs, audio_prompt_len=prompt_len)
processor.save_audio(outputs, "example_with_audio.wav")

## 5. CosyVoice 1/2/3

## 6. F5 TTS